In [ ]:
# focus only on common files
# fn to compare file name in common_files to section name
# if match, add a field to file called match_name and fill it with 1
# if not match, add a field to file called match_name and fill it with 0
# structure of comparison
# {"section_name" : {"match_name" : 1/0, "score" : 0.0}}
# score is the score of the file in the section
# compare the match_name field between two dicts
# if match_name is 1 then compare the score field
# if score is equal then add a field called model and fill it with the model name
# if score is not equal then add a field called model and fill it with the model name of the file with the highest score
# count each of model name (unique values)
# the one with higher count is more accurate

In [2]:
import json
import pandas as pd

In [3]:
def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [4]:
def post_process(data):
    # remove the only_one_model key from each section since its empty anyway
    for section in data:
        del data[section]["only_one_model"]
    # convert each list in the common_files key to a key, value pair
    for key in data.keys():
        data[key] = {file[0]: file[1] for file in data[key]["common_files"]}
    return data

In [5]:
# for section in list of sections
# for file name in section
# remove suffix ".php" 
# if section name == file name
# add a field called retrieved and fill it with 1
# add a field called score and fill it with the score
# if section name != file name
# add a field called retrieved and fill it with 0
# add a field called score and fill it with the 0.0
def create_eval_dict(data):
    eval = {}
    for section in data:
        eval[section] = {}
        target_file = section + ".php"
        if target_file in data[section].keys():
            eval[section]["retrieved"] = 1
            eval[section]["score"] = data[section][target_file]
        else:
            eval[section]["retrieved"] = 0
            eval[section]["score"] = 0.0
    return eval


In [6]:
def convert_dict_df(data):
    # convert result dictionary into a dataframe with section names, files names, and scores as columns
    data_df = pd.DataFrame([
        {"section_name": section, "retrieved": metrics["retrieved"], "score": metrics["score"]}
        for section, metrics in data.items()
    ])
    return data_df

In [7]:
def get_retrieve_accuracy(eval_df):
    ones_count = int(eval_df[eval_df["retrieved"] == 1]["retrieved"].value_counts().iloc[0])
    print(f"Count of ones: {ones_count}")
    row_count = eval_df.shape[0]
    print(f"Count total observations: {row_count}")
    return ones_count / row_count * 100

In [8]:
openai_data = read_json_file("section_result_openai_embeddings.json")
openai_data = post_process(openai_data)
openai_eval = create_eval_dict(openai_data)
openai_eval_df = convert_dict_df(openai_eval)
openai_retrieve_accuracy = get_retrieve_accuracy(openai_eval_df)
openai_retrieve_accuracy

Count of ones: 42
Count total observations: 69


60.86956521739131

In [9]:
ada_specter_data = read_json_file("section_result_azure_embeddings.json")
ada_specter_data = post_process(ada_specter_data)
ada_specter_eval = create_eval_dict(ada_specter_data)
ada_specter_eval_df = convert_dict_df(ada_specter_eval)
ada_specter_retrieve_accuracy = get_retrieve_accuracy(ada_specter_eval_df)
ada_specter_retrieve_accuracy

Count of ones: 45
Count total observations: 69


65.21739130434783

In [10]:
large_3_specter_data = read_json_file("section_result_text_embeddings_large_3.json")
large_3_specter_data = post_process(large_3_specter_data)
large_3_specter_eval = create_eval_dict(large_3_specter_data)
large_3_specter_eval_df = convert_dict_df(large_3_specter_eval)
large_3_specter_retrieve_accuracy = get_retrieve_accuracy(large_3_specter_eval_df)
large_3_specter_retrieve_accuracy

Count of ones: 46
Count total observations: 69


66.66666666666666

In [14]:
ada_data = read_json_file("section_result_ada_embedding_only.json")
ada_data = post_process(ada_data)
ada_eval = create_eval_dict(ada_data)
ada_eval_df = convert_dict_df(ada_eval)
ada_retrieve_accuracy = get_retrieve_accuracy(ada_eval_df)
ada_retrieve_accuracy

Count of ones: 55
Count total observations: 69


79.71014492753623

In [13]:
large_3_data = read_json_file("section_result_text_embeddings_large_3_only.json")
large_3_data = post_process(large_3_data)
large_3_eval = create_eval_dict(large_3_data)
large_3_eval_df = convert_dict_df(large_3_eval)
large_3_retrieve_accuracy = get_retrieve_accuracy(large_3_eval_df)
large_3_retrieve_accuracy

Count of ones: 58
Count total observations: 69


84.05797101449275